# Tableau Dashboard Data for Retail WoW Leaderboard Stats

- Goal is to build a Tableau dashboard for looking at achievement leaderboard stats (and leaderboard guildmates)
- Original data is from the Subscript web app (in the insight_data_science repo)
- New data includes detailed profile information including class and active spec

In [1]:
import os
import pandas as pd
import google_sheets as gs
import wow_retail_api as wow
import configparser as cp
import json
import file_concatenator as fc
import requests

In [2]:
def get_access_token (blizzard_key, blizzard_secret):
    """Get access token to access the blizzard API"""
    r = requests.post('https://us.battle.net/oauth/token', data={'grant_type': 'client_credentials'},
                  auth=(blizzard_key, blizzard_secret))
    unpacked = json.loads(r.text)
    access_token = unpacked['access_token']
    return access_token

In [3]:
def get_wow_profile (realm, player, token):
    """Retrievesthe public profile for a player using the Blizzard API"""
    url = 'https://us.api.blizzard.com/profile/wow/character/' + realm \
          + '/' + player + '?namespace=profile-us&locale=en_US&access_token=' + token
    try:
        r = requests.get(url)
    except:
        pass
    unpacked = json.loads(r.text)
    try:
        row = dict(id = unpacked['id'], name = unpacked['name'], gender = unpacked['gender']['name'],
          faction = unpacked['faction']['name'], race = unpacked['race']['name'],
          character_class = unpacked['character_class']['name'],
          active_spec = unpacked['active_spec']['name'], realm = unpacked['realm']['slug'],
          guild = unpacked['guild']['name'], level = unpacked['level'],
          achievement_points = unpacked['achievement_points'],
          last_login = unpacked['last_login_timestamp'],
          average_item_level = unpacked['average_item_level'],
          equipped_item_level = unpacked['equipped_item_level'])
        return row
    except:
        pass

### Get API credentials

In [4]:
f_config = '../wow_config.ini'
config = cp.ConfigParser()
config.read(f_config)
key = config.get('client', 'blizzard')
secret = config.get('secret', 'blizzard')
token = get_access_token (key, secret)

### Get Character List
- Recycling the leaderboards and their guildmates from the Subscript project

In [5]:
df = pd.read_csv(os.path.join('data', 'final_time_stats.csv'))
df = df.dropna()
unnamed = [c for c in df.columns.values if 'unnamed' in c.lower()]
df = df.drop(unnamed, axis = 1)
df['id'] = df.player + '_' + df.realm
df

,player,realm,gear_score,last_login,time_since_login,2014-01,2015-01,2016-01,2017-01,2018-01,...,2014-12,2015-12,2016-12,2017-12,2018-12,2019-12,2020-12,engagement,status,id
0,runeknight,whisperwind,437.0,2020-05-31,5 days 00:00:00.000000000,0.0,0.0,4.0,0.0,5.0,...,7.0,7.0,0.0,1.0,2.0,3.0,0.0,0.0,active,runeknight_whisperwind
1,chrichton,whisperwind,451.0,2020-06-04,1 days 00:00:00.000000000,0.0,4.0,1.0,1.0,6.0,...,23.0,6.0,7.0,2.0,1.0,3.0,0.0,0.0,active,chrichton_whisperwind
2,shìfted,whisperwind,447.0,2020-06-07,-2 days +00:00:00.000000000,0.0,10.0,1.0,0.0,6.0,...,53.0,16.0,1.0,0.0,3.0,48.0,0.0,0.0,active,shìfted_whisperwind
3,láwdamercy,whisperwind,329.0,2020-06-07,-2 days +00:00:00.000000000,0.0,7.0,0.0,0.0,10.0,...,21.0,10.0,2.0,0.0,2.0,10.0,0.0,0.0,active,láwdamercy_whisperwind
4,portsforsale,whisperwind,268.0,2019-10-23,226 days 00:00:00.000000000,3.0,10.0,2.0,6.0,1.0,...,19.0,4.0,7.0,0.0,4.0,1.0,0.0,1.0,risk,portsforsale_whisperwind
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169449,xanarian,arathor,416.0,2019-09-25,254 days 00:00:00.000000000,2.0,0.0,1.0,5.0,0.0,...,2.0,0.0,3.0,4.0,6.0,35.0,0.0,2.0,lapsed,xanarian_arathor
169450,trails,arathor,435.0,2019-10-17,232 days 00:00:00.000000000,2.0,0.0,1.0,5.0,0.0,...,2.0,0.0,3.0,4.0,6.0,35.0,0.0,1.0,risk,trails_arathor
169451,teyvarix,drenden,476.0,2020-06-10,-5 days +00:00:00.000000000,2.0,0.0,1.0,5.0,0.0,...,2.0,0.0,3.0,4.0,3.0,16.0,0.0,0.0,active,teyvarix_drenden
169452,daxus,drenden,465.0,2020-06-10,-5 days +00:00:00.000000000,2.0,0.0,1.0,5.0,0.0,...,2.0,0.0,3.0,4.0,3.0,13.0,0.0,0.0,active,daxus_drenden


### Get character basics

In [6]:
df_profile = pd.DataFrame()
count = 0

for index, row in df.iloc[:500].iterrows():
    try:
        row_profile = get_wow_profile(row.realm, row.player, token)
        df_profile = df_profile.append(row_profile, ignore_index = True)  
        if count % 500 == 0:
            df_profile.to_csv(os.path.join('data','profile','profile' + '_'+ str(count) + '.csv'))
            df_profile = pd.DataFrame()
        count += 1
        print(count, end = ' ')
    except:
        pass
    
df_profile

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

,achievement_points,active_spec,average_item_level,character_class,equipped_item_level,faction,gender,guild,id,last_login,level,name,race,realm
0,10665.0,Beast Mastery,452.0,Hunter,452.0,Horde,Male,Scotch and Soda,108852375.0,1.599147e+12,120.0,Chrichton,Vulpera,whisperwind
1,10665.0,Restoration,451.0,Druid,450.0,Horde,Male,Scotch and Soda,109071045.0,1.596777e+12,120.0,Shìfted,Highmountain Tauren,whisperwind
2,10665.0,Discipline,329.0,Priest,329.0,Horde,Male,Scotch and Soda,109133210.0,1.593437e+12,120.0,Láwdamercy,Tauren,whisperwind
3,12985.0,Fire,269.0,Mage,268.0,Horde,Female,Scotch and Soda,109579770.0,1.571866e+12,120.0,Portsforsale,Goblin,whisperwind
4,18075.0,Frost,471.0,Death Knight,468.0,Horde,Male,Explicitly,110371779.0,1.599153e+12,120.0,Raintuzk,Blood Elf,whisperwind
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,10535.0,Restoration,472.0,Druid,469.0,Alliance,Female,BlameRoll,191960165.0,1.599006e+12,120.0,Hochiminh,Night Elf,proudmoore
455,23405.0,Assassination,478.0,Rogue,475.0,Alliance,Male,BlameRoll,192133164.0,1.597781e+12,120.0,Ishgar,Night Elf,proudmoore
456,11535.0,Retribution,439.0,Paladin,439.0,Alliance,Male,BlameRoll,192180874.0,1.597772e+12,120.0,Yunghao,Lightforged Draenei,proudmoore
457,18515.0,Restoration,377.0,Druid,377.0,Alliance,Male,BlameRoll,192192091.0,1.598653e+12,120.0,Daddyxeo,Kul Tiran,proudmoore


### Concatenate Files

In [ ]:
df_profile = fc.concat(os.path.join('data','profile'))

### Clean Profile Details to match original

In [7]:
df_profile.name = [c.lower() for c in df_profile.name.values]
df_profile.realm = [c.lower() for c in df_profile.realm.values]
df_profile.realm = [c.replace(' ', '_') for c in df_profile.realm.values]
df_profile['id'] = df_profile.name + '_' + df_profile.realm
df_profile

,achievement_points,active_spec,average_item_level,character_class,equipped_item_level,faction,gender,guild,id,last_login,level,name,race,realm
0,10665.0,Beast Mastery,452.0,Hunter,452.0,Horde,Male,Scotch and Soda,chrichton_whisperwind,1.599147e+12,120.0,chrichton,Vulpera,whisperwind
1,10665.0,Restoration,451.0,Druid,450.0,Horde,Male,Scotch and Soda,shìfted_whisperwind,1.596777e+12,120.0,shìfted,Highmountain Tauren,whisperwind
2,10665.0,Discipline,329.0,Priest,329.0,Horde,Male,Scotch and Soda,láwdamercy_whisperwind,1.593437e+12,120.0,láwdamercy,Tauren,whisperwind
3,12985.0,Fire,269.0,Mage,268.0,Horde,Female,Scotch and Soda,portsforsale_whisperwind,1.571866e+12,120.0,portsforsale,Goblin,whisperwind
4,18075.0,Frost,471.0,Death Knight,468.0,Horde,Male,Explicitly,raintuzk_whisperwind,1.599153e+12,120.0,raintuzk,Blood Elf,whisperwind
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,10535.0,Restoration,472.0,Druid,469.0,Alliance,Female,BlameRoll,hochiminh_proudmoore,1.599006e+12,120.0,hochiminh,Night Elf,proudmoore
455,23405.0,Assassination,478.0,Rogue,475.0,Alliance,Male,BlameRoll,ishgar_proudmoore,1.597781e+12,120.0,ishgar,Night Elf,proudmoore
456,11535.0,Retribution,439.0,Paladin,439.0,Alliance,Male,BlameRoll,yunghao_proudmoore,1.597772e+12,120.0,yunghao,Lightforged Draenei,proudmoore
457,18515.0,Restoration,377.0,Druid,377.0,Alliance,Male,BlameRoll,daddyxeo_proudmoore,1.598653e+12,120.0,daddyxeo,Kul Tiran,proudmoore


### Upload to Google Sheets

In [8]:
# Connect to google drive
service = gs.google_connect()

In [9]:
# Make a new sheet for the profile data
sheet_id = gs.create_sheets (service, 'wow_profiles')

In [10]:
# Write detailed profile data to the google sheet
gs.write_sheet(service, sheet_id, df_profile, 'Sheet1!A:ZZZ')

Sheet successfully Updated


In [12]:
df.iloc[:500].to_csv(os.path.join('data','original_profile_data.csv'))
            

In [17]:
df_join = df_profile.merge(df.iloc[:500], on = 'id')
df_join.to_csv(os.path.join('data','joined_profile_data.csv'))


In [18]:
# Write the joined achievement stats file to google sheets
sheet_id = gs.create_sheets (service, 'wow_profile_joined')
gs.write_sheet(service, sheet_id, df_join.iloc[:500,:], 'Sheet1!A:ZZZ')

Sheet successfully Updated
